In [173]:
import pandas as pd

import requests

from xml.dom import minidom

import re

import sys

import numpy as np

import json

import pickle

sys.path.append('..')
from src.features.build_features import syns, sep_urls, check_paren, repo_label
from src.data.make_dataset import return_passages, test_suitability

Here, I'm trying to get data availability statements for the papers we are working with (see, eg. https://www.datacatalogcollaborationproject.org/dccp-blog/2018/10/31/finding-data-elsewhere-some-experiments-with-pmc and subsequent posts at the same place). Unfortunately, it seems as though a goodly number of the papers are unavailable through the pmc api:

https://www.ncbi.nlm.nih.gov/pmc/tools/oai/

In the end, I'm able to get data for just under 10k of the nearly 60k papers

In [174]:
nimh_papers = pd.read_csv('/data/riddleta/data_sharing_reuse/external/nimh_papers.csv')

In [175]:
base_path = "https://www.ncbi.nlm.nih.gov/pmc/oai/oai.cgi?verb=GetRecord&identifier=oai:pubmedcentral.nih.gov:" 
out_dict = dict.fromkeys(nimh_papers.pmcid.tolist())
misses = []
progress = [i for i in range(0, 57000, 2500)]

for number, i in enumerate(nimh_papers.pmcid.tolist()):
    if number in progress:
        print(number)
    pth = base_path + str(i) + "&metadataPrefix=pmc"
    r = requests.get(pth)
    xmldoc = minidom.parseString(r.text)
    itemlist = xmldoc.getElementsByTagName('sec')
    sections = []
    if len(sections) == 0:
        misses.append(i)
    for s in itemlist:
        if 'sec-type' in s.attributes:
            sections.append(s.attributes['sec-type'].value)
            
    out_dict[i] = sections

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000


In [176]:
len(misses)

57771

In [180]:
tempdf = pd.DataFrame.from_dict(out_dict, orient='index')

In [181]:
out_dict[3088774]

['methods', 'supplementary-material']

In [194]:
melted_sections = tempdf.reset_index().melt(id_vars=['index'])

In [200]:
melted_sections.rename(columns={'index':'pmcid', 
                                'variable':'section_number', 
                                'value':'section_name'}, inplace=True)

In [201]:
melted_sections.head()

,pmcid,section_number,section_name
0,3369267,0,None
1,3541781,0,None
2,2851144,0,None
3,2851143,0,None
4,3859342,0,None


In [202]:
melted_sections.pmcid.unique().shape

(57692,)

In [213]:
melted_sections_valid = melted_sections[~pd.isna(melted_sections.section_name)]

In [216]:
melted_sections_valid.pmcid.unique().shape

(9853,)